In [24]:
from transformers import AutoTokenizer
import numpy as np

# Initialize the tokenizer
# tokenizer_list = 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize each word and count the number of tokens
num_tokens = [len(tokenizer.tokenize(word)) for word in words]

# Calculate the average number of tokens per word
average_tokens = np.mean(num_tokens)

print(f"Average number of tokens per word: {average_tokens}")

Average number of tokens per word: 1.25


In [28]:
import pickle
# with open('/Noise Dics/wiki_typos_en_p320000_l9368.pkl', 'rb') as f:
#     ds_t = pickle.load(f)

def get_f_r(models_list, wt_dics_path):
    fr_results = {}
        
    for model_name in models_list:
        tokenizer = AutoTokenizer.from_pretrained(models_list[model_name])
        for lang in wt_dics_path:
            with open(wt_dics_path[lang], 'rb') as f:
                ds_t = pickle.load(f)
            noisy_words = [t[0] for t in ds_t]
            clean_words = [t[1] for t in ds_t]
            
            num_tokens_clean = [len(tokenizer.tokenize(word)) for word in clean_words]
            num_tokens_noisy = [len(tokenizer.tokenize(word)) for word in noisy_words]
            
            average_tokens_clean = np.mean(num_tokens_clean)
            average_tokens_noisy = np.mean(num_tokens_noisy)
            if model_name not in fr_results:
                fr_results[model_name] = {}
            fr_results[model_name][lang] = {
                'average_tokens_clean': average_tokens_clean,
                'average_tokens_noisy': average_tokens_noisy
            }
    return fr_results


In [29]:
models_list = {
    'xlmr' : 'xlm-roberta-base',
    'mbert' : 'google-bert/bert-base-multilingual-cased',
    'falcon7b' : 'tiiuae/falcon-7b',
    'mt5': 'google/mt5-small',
    'bloom7b' : 'bigscience/bloom-7b1'
}

wt_dics_path = {
    'en' : '/Noise Dics/wiki_typos_en_p320000_l9368.pkl',
    'de' : '/Noise Dics/wiki_typos_de_p320000_l15023.pkl',
    'es' : '/Noise Dics/wiki_typos_es_p320000_l8212.pkl',
    'fr' : '/Noise Dics/wiki_typos_fr_p320000_l14872.pkl',
    'hi' : '/Noise Dics/wiki_typos_hi_p320000_l3277.pkl',
    'tr' : '/Noise Dics/wiki_typos_tr_p320000_l4064.pkl',
}


res=get_f_r(models_list, wt_dics_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/s6amalia/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [40]:
import pandas as pd
df = pd.DataFrame.from_dict({(i,j): res[i][j] 
                           for i in res.keys() 
                           for j in res[i].keys()},
                       orient='index')

df.round(2)

average_tokens_clean  average_tokens_noisy
xlmr     en                  2.00                  2.10
         de                  2.38                  2.54
         es                  2.00                  2.10
         fr                  2.10                  2.18
         hi                  1.53                  1.56
         tr                  2.14                  2.32
mbert    en                  1.97                  2.14
         de                  2.32                  2.57
         es                  1.97                  2.14
         fr                  1.98                  2.17
         hi                  1.62                  1.68
         tr                  2.53                  2.67
falcon7b en                  2.74                  2.63
         de                  3.09                  3.13
         es                  2.74                  2.63
         fr                  2.52                  2.53
         hi                  2.59                  2.43
         tr                  3.64                  3.54
mt5      en                  2.14                  2.16
         de                  2.38                  2.51
         es                  2.14                  2.16
         fr                  2.17                  2.21
         hi                  1.70                  1.68
         tr                  2.28                  2.38
bloom7b  en                  2.28                  2.33
         de                  3.12                  3.14
         es                  2.28                  2.33
         fr                  2.26                  2.33
         hi                  1.63                  1.64
         tr                  3.17                  3.11

In [41]:
df = df.drop(('bloom7b', 'de'))
df = df.drop(('bloom7b', 'tr'))
df = df.drop(('falcon7b', 'tr'))
df = df.drop(('falcon7b', 'hi'))
df

average_tokens_clean  average_tokens_noisy
xlmr     en              1.998173              2.104238
         de              2.384943              2.542635
         es              1.998173              2.104238
         fr              2.095212              2.180608
         hi              1.525175              1.563930
         tr              2.137549              2.320128
mbert    en              1.972114              2.141744
         de              2.324236              2.565799
         es              1.972114              2.141744
         fr              1.982249              2.165546
         hi              1.624352              1.676228
         tr              2.528297              2.670522
falcon7b en              2.737214              2.633707
         de              3.094322              3.129601
         es              2.737214              2.633707
         fr              2.517819              2.533015
mt5      en              2.143327              2.157453
         de              2.384544              2.508354
         es              2.143327              2.157453
         fr              2.172203              2.206764
         hi              1.700336              1.680806
         tr              2.275098              2.376969
bloom7b  en              2.275329              2.325012
         es              2.275329              2.325012
         fr              2.262171              2.330689
         hi              1.625267              1.641440

In [42]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk
import random
lang = 'en'
# /xnli_WikiTypos/hi_noisy.hf
ds_typo = load_from_disk('/xnli_WikiTypos/'+lang+'_noisy.hf') 
ds_orig = load_dataset('xnli', lang,split= 'test')


In [47]:
df=pd.DataFrame(ds_typo)
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
df['num_tokens'] = df['premise'].apply(lambda x: len(tokenizer.tokenize(x)))
average_tokens = df['num_tokens'].mean()
print('Average number of tokens:', average_tokens)


Average number of tokens: 26.812774451097805


In [48]:
df=pd.DataFrame(ds_orig)
df['num_tokens'] = df['premise'].apply(lambda x: len(tokenizer.tokenize(x)))
average_tokens = df['num_tokens'].mean()
print('Average number of tokens:', average_tokens)

Average number of tokens: 25.019161676646707


In [55]:
ds_orig = load_dataset('wikiann','en', split='test')
ds_typo = load_from_disk(f'/wikiann_nlpaug/{lang}_noisy.hf')

In [59]:
df=pd.DataFrame(ds_orig)

df['num_tokens'] = df['tokens'].apply(lambda x: len(tokenizer.tokenize(x,is_split_into_words=True)))
average_tokens = df['num_tokens'].mean()
print('Average number of tokens:', average_tokens)

Average number of tokens: 10.9327


In [60]:
df=pd.DataFrame(ds_typo)
df['num_tokens'] = df['tokens'].apply(lambda x: len(tokenizer.tokenize(x,is_split_into_words=True)))
average_tokens = df['num_tokens'].mean()
print('Average number of tokens:', average_tokens)

Average number of tokens: 12.1382


In [66]:
langs = ['en', 'de', 'es', 'fr','tr']
fr_wikiann={}
for model_name in models_list:
    if "bloom7b" in model_name:

        tokenizer = AutoTokenizer.from_pretrained(models_list[model_name],add_prefix_space=True)
    else:
        tokenizer = AutoTokenizer.from_pretrained(models_list[model_name])
    print(model_name)
    for lang in langs:
        ds_orig = load_dataset('wikiann',lang, split='test')
        ds_typo = load_from_disk(f'/wikiann_nlpaug/{lang}_noisy.hf')

        df_orig=pd.DataFrame(ds_orig)
        df_typo=pd.DataFrame(ds_typo)

        df_orig['num_tokens'] = df_orig['tokens'].apply(lambda x: len(tokenizer.tokenize(x,is_split_into_words=True)))
        df_typo['num_tokens'] = df_typo['tokens'].apply(lambda x: len(tokenizer.tokenize(x,is_split_into_words=True)))

        average_tokens_orig = df_orig['num_tokens'].mean()
        average_tokens_typo = df_typo['num_tokens'].mean()
        if model_name not in fr_wikiann:
            fr_wikiann[model_name] = {}
        fr_wikiann[model_name][lang] = {
                'average_tokens_clean': average_tokens_orig,
                'average_tokens_noisy': average_tokens_typo
            }
fr_wikiann       

xlmr
mbert
falcon7b


/home/s6amalia/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


mt5
bloom7b


{'xlmr': {'en': {'average_tokens_clean': 12.0604,
   'average_tokens_noisy': 13.0238},
  'de': {'average_tokens_clean': 15.7261, 'average_tokens_noisy': 16.8417},
  'es': {'average_tokens_clean': 10.4835, 'average_tokens_noisy': 11.3251},
  'fr': {'average_tokens_clean': 11.4272, 'average_tokens_noisy': 12.171},
  'tr': {'average_tokens_clean': 13.285, 'average_tokens_noisy': 14.3105}},
 'mbert': {'en': {'average_tokens_clean': 10.9327,
   'average_tokens_noisy': 12.1382},
  'de': {'average_tokens_clean': 14.5093, 'average_tokens_noisy': 15.8823},
  'es': {'average_tokens_clean': 10.7219, 'average_tokens_noisy': 11.7617},
  'fr': {'average_tokens_clean': 10.6603, 'average_tokens_noisy': 11.6285},
  'tr': {'average_tokens_clean': 14.9001, 'average_tokens_noisy': 15.9295}},
 'falcon7b': {'en': {'average_tokens_clean': 12.5135,
   'average_tokens_noisy': 13.4207},
  'de': {'average_tokens_clean': 19.1397, 'average_tokens_noisy': 19.8354},
  'es': {'average_tokens_clean': 12.603, 'average_

In [81]:
langs = ['en', 'de', 'es', 'fr','hi','tr']
fr_xnli={}
for model_name in models_list:
    tokenizer = AutoTokenizer.from_pretrained(models_list[model_name])
    for lang in langs:
        ds_orig = load_dataset('xnli',lang, split='test')
        ds_typo = load_from_disk('/xnli_WikiTypos/'+lang+'_noisy.hf') 

        df_orig=pd.DataFrame(ds_orig)
        df_typo=pd.DataFrame(ds_typo)
        df_orig['num_tokens'] = df_orig.apply(lambda row: len(tokenizer.tokenize(row['premise'])) + len(tokenizer.tokenize(row['hypothesis'])), axis=1)
        df_typo['num_tokens'] = df_typo.apply(lambda row: len(tokenizer.tokenize(row['premise'])) + len(tokenizer.tokenize(row['hypothesis'])), axis=1)
        # df_orig['num_tokens'] = df_orig['premise'].apply(lambda x: len(tokenizer.tokenize(x)))
        # df_typo['num_tokens'] = df_typo['premise'].apply(lambda x: len(tokenizer.tokenize(x)))

        average_tokens_orig = df_orig['num_tokens'].mean()
        average_tokens_typo = df_typo['num_tokens'].mean()
        if model_name not in fr_xnli:
            fr_xnli[model_name] = {}
        fr_xnli[model_name][lang] = {
                'average_tokens_clean': average_tokens_orig,
                'average_tokens_noisy': average_tokens_typo
            }
fr_xnli      

/home/s6amalia/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


{'xlmr': {'en': {'average_tokens_clean': 38.178642714570856,
   'average_tokens_noisy': 43.80499001996008},
  'de': {'average_tokens_clean': 42.189221556886224,
   'average_tokens_noisy': 48.16127744510978},
  'es': {'average_tokens_clean': 40.93692614770459,
   'average_tokens_noisy': 46.367065868263474},
  'fr': {'average_tokens_clean': 47.59760479041916,
   'average_tokens_noisy': 52.927744510978044},
  'hi': {'average_tokens_clean': 45.70858283433134,
   'average_tokens_noisy': 48.557884231536924},
  'tr': {'average_tokens_clean': 37.84790419161676,
   'average_tokens_noisy': 43.88882235528942}},
 'mbert': {'en': {'average_tokens_clean': 37.10758483033932,
   'average_tokens_noisy': 39.88063872255489},
  'de': {'average_tokens_clean': 44.03912175648703,
   'average_tokens_noisy': 45.89101796407186},
  'es': {'average_tokens_clean': 42.29780439121757,
   'average_tokens_noisy': 44.21776447105788},
  'fr': {'average_tokens_clean': 47.62295409181637,
   'average_tokens_noisy': 49.5121

In [76]:
langs = ['en', 'de', 'es', 'fr','hi','tr']
fr_snips={}
for model_name in models_list:
    tokenizer = AutoTokenizer.from_pretrained(models_list[model_name])
    for lang in langs:
        if lang == 'en':
            ds_orig = load_dataset('benayas/snips')
        else:
            ds_orig = load_from_disk("/snips_dataset/snips_" + lang + ".hf")
        

        ds_typo = load_from_disk(f"/snips_WikiTypos/{lang}_noisy.hf")
        
        # ds_orig = load_dataset('xnli',lang, split='test')
        # ds_typo = load_from_disk('/xnli_WikiTypos/'+lang+'_noisy.hf') 

        df_orig=pd.DataFrame(ds_orig['test'])
        df_typo=pd.DataFrame(ds_typo)

        df_orig['num_tokens'] = df_orig['text'].apply(lambda x: len(tokenizer.tokenize(x)))
        df_typo['num_tokens'] = df_typo['text'].apply(lambda x: len(tokenizer.tokenize(x)))

        average_tokens_orig = df_orig['num_tokens'].mean()
        average_tokens_typo = df_typo['num_tokens'].mean()
        if model_name not in fr_snips:
            fr_snips[model_name] = {}
        fr_snips[model_name][lang] = {
                'average_tokens_clean': average_tokens_orig,
                'average_tokens_noisy': average_tokens_typo
            }
fr_snips      

/home/s6amalia/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


{'xlmr': {'en': {'average_tokens_clean': 11.812142857142858,
   'average_tokens_noisy': 12.601428571428572},
  'de': {'average_tokens_clean': 14.130714285714285,
   'average_tokens_noisy': 15.077857142857143},
  'es': {'average_tokens_clean': 14.215,
   'average_tokens_noisy': 15.174285714285714},
  'fr': {'average_tokens_clean': 14.919285714285714,
   'average_tokens_noisy': 15.985714285714286},
  'hi': {'average_tokens_clean': 15.365,
   'average_tokens_noisy': 15.84142857142857},
  'tr': {'average_tokens_clean': 13.004285714285714,
   'average_tokens_noisy': 14.25}},
 'mbert': {'en': {'average_tokens_clean': 11.486428571428572,
   'average_tokens_noisy': 12.155714285714286},
  'de': {'average_tokens_clean': 14.115,
   'average_tokens_noisy': 14.723571428571429},
  'es': {'average_tokens_clean': 14.135714285714286,
   'average_tokens_noisy': 14.787142857142857},
  'fr': {'average_tokens_clean': 14.732857142857142,
   'average_tokens_noisy': 15.489285714285714},
  'hi': {'average_toke

In [78]:
import pandas as pd
df_snips = pd.DataFrame.from_dict({(i,j): fr_snips[i][j] 
                           for i in fr_snips.keys() 
                           for j in fr_snips[i].keys()},
                       orient='index')

df_snips.round(2)

average_tokens_clean  average_tokens_noisy
xlmr     en                 11.81                 12.60
         de                 14.13                 15.08
         es                 14.22                 15.17
         fr                 14.92                 15.99
         hi                 15.36                 15.84
         tr                 13.00                 14.25
mbert    en                 11.49                 12.16
         de                 14.12                 14.72
         es                 14.14                 14.79
         fr                 14.73                 15.49
         hi                 21.20                 21.45
         tr                 15.65                 16.14
falcon7b en                 11.58                 12.47
         de                 16.13                 17.07
         es                 16.22                 17.28
         fr                 16.00                 17.01
         hi                 72.67                 72.67
         tr                 20.72                 21.89
mt5      en                 12.52                 13.42
         de                 15.11                 16.20
         es                 15.97                 17.15
         fr                 17.13                 18.05
         hi                 19.98                 20.51
         tr                 14.41                 15.94
bloom7b  en                 11.00                 11.63
         de                 16.58                 16.97
         es                 13.03                 13.73
         fr                 12.99                 13.70
         hi                 14.03                 14.44
         tr                 17.64                 18.02

In [79]:
df_wikiann = pd.DataFrame.from_dict({(i,j): fr_wikiann[i][j] 
                           for i in fr_wikiann.keys() 
                           for j in fr_wikiann[i].keys()},
                       orient='index')

df_wikiann.round(2)

average_tokens_clean  average_tokens_noisy
xlmr     en                 12.06                 13.02
         de                 15.73                 16.84
         es                 10.48                 11.33
         fr                 11.43                 12.17
         tr                 13.28                 14.31
mbert    en                 10.93                 12.14
         de                 14.51                 15.88
         es                 10.72                 11.76
         fr                 10.66                 11.63
         tr                 14.90                 15.93
falcon7b en                 12.51                 13.42
         de                 19.14                 19.84
         es                 12.60                 13.16
         fr                 12.57                 13.15
         tr                 20.80                 21.25
mt5      en                 12.79                 13.68
         de                 16.92                 17.82
         es                 11.68                 12.33
         fr                 12.32                 12.98
         tr                 15.18                 15.94
bloom7b  en                 10.64                 11.67
         de                 17.20                 18.01
         es                  9.26                 10.19
         fr                  9.59                 10.46
         tr                 17.71                 18.28

In [82]:
df_xnli = pd.DataFrame.from_dict({(i,j): fr_xnli[i][j] 
                           for i in fr_xnli.keys() 
                           for j in fr_xnli[i].keys()},
                       orient='index')

df_xnli.round(2)

average_tokens_clean  average_tokens_noisy
xlmr     en                 38.18                 43.80
         de                 42.19                 48.16
         es                 40.94                 46.37
         fr                 47.60                 52.93
         hi                 45.71                 48.56
         tr                 37.85                 43.89
mbert    en                 37.11                 39.88
         de                 44.04                 45.89
         es                 42.30                 44.22
         fr                 47.62                 49.51
         hi                 61.91                 63.31
         tr                 48.56                 50.23
falcon7b en                 34.99                 41.38
         de                 49.42                 55.12
         es                 47.02                 52.50
         fr                 49.33                 54.70
         hi                232.85                234.02
         tr                 73.16                 77.77
mt5      en                 42.08                 48.05
         de                 48.18                 53.96
         es                 49.51                 54.67
         fr                 57.16                 62.41
         hi                 64.44                 66.69
         tr                 44.35                 50.05
bloom7b  en                 34.41                 37.38
         de                 54.02                 55.16
         es                 37.34                 39.59
         fr                 38.85                 41.19
         hi                 42.79                 44.25
         tr                 62.50                 63.50